In [ ]:
from google.colab import drive
drive.mount('./Content')

Mounted at ./Content


In [ ]:
!pip install easyocr

In [ ]:
!pip install ultralytics

In [13]:
import csv
import cv2
import easyocr
import numpy as np
import os
from ultralytics import YOLO

# Inisialisasi model YOLOv8
model = YOLO('/content/drive/MyDrive/Colab Notebooks/Parking System/output model/100epoch_91%acc_NewDataset.pt')

# Inisialisasi EasyOCR
reader = easyocr.Reader(['en'])

# Membuka video input
input_video_path = '/content/drive/MyDrive/Colab Notebooks/Parking System/testing data/Agus 3.mp4'
cap = cv2.VideoCapture(input_video_path)
kernel_sharpening = np.array([[0,  1,  0],[1, -4,  1],[0,  1,  0]])

# Mendapatkan direktori video
video_directory = os.path.dirname(input_video_path)

# Membuat nama file CSV untuk menyimpan hasil OCR
csv_file_path = os.path.join(video_directory, 'ocr_results19(Tes).csv')
csv_file = open(csv_file_path, 'w', newline='')
csv_writer = csv.writer(csv_file)

# Menyimpan video output
output_video_path = '/content/drive/MyDrive/Colab Notebooks/Parking System/output testing/NewDatasetOutput_video19(Tes).mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Format kodek video
width = int(cap.get(3))
height = int(cap.get(4))
output_video = cv2.VideoWriter(output_video_path, fourcc, 20.0, (width, height))  # Resolusi sesuai dengan video input

threshold = 0.7  # Ambang batas untuk deteksi objek

# Inisialisasi penghitung frame
count = 0

# import re

# def correct_text(text):
#     pattern = r'^[a-zA-Z]{2}\d{4}[a-zA-Z]{2}$'
#     if re.match(pattern, text):
#         return text
#     else:
#         # Menghapus titik dan koma
#         text = re.sub(r'[.,]', '', text)
#         # Menghapus karakter spasi
#         text = re.sub(r'\s', '', text)
#         # Mengambil 8 karakter pertama
#         text = text[:8]
#         text = text[:2] + text[2:6] + text[6:]
#         # Mengganti karakter yang tidak sesuai dengan pola dengan karakter kosong
#         text = re.sub(r'^a-zA-Z', '', text)  # Huruf di awal
#         text = re.sub(r'\D(?<=\d{4})', '', text)  # Angka di tengah
#         text = re.sub(r'^a-zA-Z', '', text)  # Huruf di akhir
#         return text

while True:
    # Membaca frame dari video input
    ret, frame = cap.read()

    if not ret:
        break

    # Mendeteksi plat nomor motor dalam frame
    results = model(frame)[0]

    # Sorting the results based on the x-coordinate
    sorted_results = sorted(results.boxes.data.tolist(), key=lambda x: x[0])

    for result in sorted_results:
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            # Extract text from bounding box using EasyOCR
            box = frame[int(y1):int(y2), int(x1):int(x2)]

            # OCR pada citra yang telah ditingkatkan
            ocr_result = reader.readtext(box)
            text = ' '.join([line[1] for line in ocr_result])

            # Hapus titik dan koma dari teks
            text = text.replace('.', '').replace(',', '')

            # Hapus karakter spasi
            text = text.replace(' ', '')

            # Ambil 8 karakter pertama dengan format yang ditentukan
            first_8_characters = text[:8]
            text = first_8_characters[:2] + first_8_characters[2:6] + first_8_characters[6:]

            # Draw bounding box and text on the original frame
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
            cv2.putText(frame, text, (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)
            # Save the image with bounding box
            cv2.imwrite('/content/drive/MyDrive/Colab Notebooks/Parking System/output testing/frame%d.jpg' % count, box)
            count += 1

            # Menyimpan hasil OCR ke file CSV
            csv_writer.writerow([text])

    # Menyimpan frame yang telah dimodifikasi ke dalam video output
    output_video.write(frame)

# Tutup video input, output, dan file CSV
cap.release()
output_video.release()
csv_file.close()
cv2.destroyAllWindows()



0: 384x640 1 Plat-Motor, 139.1ms
Speed: 2.2ms preprocess, 139.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Plat-Motor, 114.3ms
Speed: 6.5ms preprocess, 114.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Plat-Motor, 88.1ms
Speed: 2.1ms preprocess, 88.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Plat-Motor, 91.1ms
Speed: 2.1ms preprocess, 91.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Plat-Motor, 87.6ms
Speed: 3.7ms preprocess, 87.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Plat-Motor, 81.8ms
Speed: 2.4ms preprocess, 81.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Plat-Motors, 83.6ms
Speed: 2.2ms preprocess, 83.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Plat-Motors, 82.1ms
Speed: 3.4ms preprocess, 82.1ms inference, 0.6ms po

In [ ]:
# Draw bounding box and text on the original frame
cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
cv2.putText(frame, text, (int(x1), int(y1 - 10)),
cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

# Save the image with bounding box
cv2.imwrite('output_image.jpg', frame)
